<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.879 - TFM - Área 2 aula 1 - Machine Learning</p>
<p style="margin: 0; text-align:right;">Máster universitario de Ciencia de datos (<i>Data Science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Adquisición de datos
En este módulo se pretenden descargar las imágenes de ImageNet de las clases manzana, pera, plátano, limón, naranja y piña, para poder obtener un primer conjunto de imágenes de entrenamiento y de validación.
Por cada clase descargaremos 100 de entrenamiento y 50 de validación. Asegurando que no existe ninguna imagen que este repetida en la carpeta de entrenamiento y en la de validación.


In [5]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import urllib
import os

In [6]:
img_rows, img_cols = 32, 32 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read it into OpenCV format
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image
n_of_training_images=200#the number of training images to use
n_of_validating_images=50#the number of validating images to use

In [7]:
synsetsIds=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
synsetsNames=['apple','pear','banana','lemon','orange','pineapple']

for i in range(0,len(synsetsIds)):
    url="http://www.image-net.org/api/text/imagenet.synset.geturls.getmapping?wnid="+synsetsIds[i]
    page = requests.get(url)
    train_path="TensorFlow/workspace/images/train/"+synsetsNames[i]+"/"
    validation_path="TensorFlow/workspace/images/test/"+synsetsNames[i]+"/"
    try:
        os.makedirs(train_path)
    except FileExistsError:
        print("Directory " , train_path ,  " already exists")
    try:
        os.makedirs(validation_path)
    except FileExistsError:
        print("Directory " , validation_path ,  " already exists")
        
    #puts the content of the website into the soup variable, each url on a different line
    soup = BeautifulSoup(page.content, 'html.parser')

    str_soup=str(soup)#convert soup to string so it can be split
    type(str_soup)
    split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
    #Train data:
    for progress in range(n_of_training_images):#store all the images on a directory
        if not split_urls[progress] == None:
            try:
                I = url_to_image(split_urls[progress].split(' ', 1)[1])
                if (len(I.shape))==3: #check if the image has width, length and channels
                    save_path = train_path+str(split_urls[progress].split(' ', 1)[0])+'.jpg'#create a name of each image
                    cv2.imwrite(save_path,I)
            except:
                None
    #Validation data:
    for progress in range(n_of_validating_images):#store all the images on a directory
        if not split_urls[progress] == None:
            try:
                #get images that are different from the ones used for training
                I = url_to_image(split_urls[n_of_training_images+progress].split(' ', 1)[1])
                if (len(I.shape))==3: #check if the image has width, length and channels
                    save_path = validation_path+str(split_urls[n_of_training_images+progress].split(' ', 1)[0])+'.jpg'#create a name of each image
                    cv2.imwrite(save_path,I)
            except:
                None